**-----------VP MWV Avg Analysis---------**

In [ ]:
import re
import numpy as np

results = [
    'Total Time - Mean: 3360.4400 sec, Std Dev: 33.8138',
    'Total Time - Mean: 3657.9300 sec, Std Dev: 306.8985',
    'Total Time - Mean: 3281.7600 sec, Std Dev: 8.0610'
]

means, stds = [], []
for line in results:
    m = re.search(r'Mean:\s*([\d.]+)\s*sec,\s*Std Dev:\s*([\d.]+)', line)
    if not m:
        raise ValueError(f"No match: {line}")
    means.append(float(m.group(1)))
    stds.append(float(m.group(2)))

means = np.array(means, dtype=float)
stds  = np.array(stds,  dtype=float)

k = len(means)

combined_mean = np.mean(means)
combined_std  = np.sqrt(np.sum(stds**2)) / k
cv = (combined_std / combined_mean) * 100 if combined_mean != 0 else float('nan')

print(
    f"Combined Mean: {combined_mean:.4f} sec, "
    f"Std Dev: {combined_std:.4f} sec, CV: {cv:.2f}%"
)

Combined Mean: 3433.3767 sec, Std Dev: 102.9536 sec, CV: 3.00%


-------------------------------

**----------Exhaustive Analysis------------**

In [ ]:
import re
import numpy as np

results = [
    'Total Time - Mean: 16071.1300 sec, Std Dev: 2.6304',
    'Total Time - Mean: 18639.5150 sec, Std Dev: 8.4075',
    'Total Time - Mean: 20117.8400 sec, Std Dev: 49.8935'
]

means, stds = [], []
for line in results:
    m = re.search(r'Mean:\s*([\d.]+)\s*sec,\s*Std Dev:\s*([\d.]+)', line)
    if not m:
        raise ValueError(f"No match: {line}")
    means.append(float(m.group(1)))
    stds.append(float(m.group(2)))

means = np.array(means, dtype=float)
stds  = np.array(stds,  dtype=float)

k = len(means)

combined_mean = np.mean(means)
combined_std  = np.sqrt(np.sum(stds**2)) / k
cv = (combined_std / combined_mean) * 100 if combined_mean != 0 else float('nan')

print(
    f"Combined Mean: {combined_mean:.4f} sec, "
    f"Std Dev: {combined_std:.4f} sec, CV: {cv:.2f}%"
)

Combined Mean: 18276.1617 sec, Std Dev: 16.8884 sec, CV: 0.09%


---------------------

**----------VP Log Threshold Analysis-----------**

In [ ]:
import re
import numpy as np

results = [
    'Total Time - Mean: 185.0957 sec, Std Dev: 0.6901',
    'Total Time - Mean: 167.9660 sec, Std Dev: 0.8510',
    'Total Time - Mean: 180.6117 sec, Std Dev: 1.2522'
]

means, stds = [], []
for line in results:
    m = re.search(r'Mean:\s*([\d.]+)\s*sec,\s*Std Dev:\s*([\d.]+)', line)
    if not m:
        raise ValueError(f"No match: {line}")
    means.append(float(m.group(1)))
    stds.append(float(m.group(2)))

means = np.array(means, dtype=float)
stds  = np.array(stds,  dtype=float)

k = len(means)

combined_mean = np.mean(means)
combined_std  = np.sqrt(np.sum(stds**2)) / k
cv = (combined_std / combined_mean) * 100 if combined_mean != 0 else float('nan')

print(
    f"Combined Mean: {combined_mean:.4f} sec, "
    f"Std Dev: {combined_std:.4f} sec, CV: {cv:.2f}%"
)

Combined Mean: 177.8911 sec, Std Dev: 0.5546 sec, CV: 0.31%


---

**-----------VP Evo Alg-------------**

In [ ]:
import re
import numpy as np

results = [
    'Total Time - Mean: 166.2218 sec, Std Dev: 0.5024',
    'Total Time - Mean: 159.4459 sec, Std Dev: 1.0144',
    'Total Time - Mean: 173.8477 sec, Std Dev: 1.1159'
]

means, stds = [], []
for line in results:
    m = re.search(r'Mean:\s*([\d.]+)\s*sec,\s*Std Dev:\s*([\d.]+)', line)
    if not m:
        raise ValueError(f"No match: {line}")
    means.append(float(m.group(1)))
    stds.append(float(m.group(2)))

means = np.array(means, dtype=float)
stds  = np.array(stds,  dtype=float)

k = len(means)

combined_mean = np.mean(means)
combined_std  = np.sqrt(np.sum(stds**2)) / k
cv = (combined_std / combined_mean) * 100 if combined_mean != 0 else float('nan')

print(
    f"Combined Mean: {combined_mean:.4f} sec, "
    f"Std Dev: {combined_std:.4f} sec, CV: {cv:.2f}%"
)

Combined Mean: 166.5051 sec, Std Dev: 0.5298 sec, CV: 0.32%


--------------------

### Comparison between static-log and lamarckian
---

In [ ]:
import re
import math
from typing import Tuple, Optional

def parse_stats(s: str) -> Tuple[float, float]:
    mean_match = re.search(r"Mean:\s*([0-9.]+)", s)
    std_match  = re.search(r"Std Dev:\s*([0-9.]+)", s)

    if mean_match is None or std_match is None:
        raise ValueError(f"Could not parse stats from: {s}")

    mean = float(mean_match.group(1))
    std  = float(std_match.group(1))
    return mean, std


def compare_policies(static_stats: str,
                     lamarckian_stats: str,
                     n_static: Optional[int] = None,
                     n_lam: Optional[int] = None):
    """
    Keeps the original outputs, and adds delta_se if run counts are provided.

    delta_std = sqrt(sd_s^2 + sd_l^2)  (SD of difference across random runs)
    delta_se  = sqrt(sd_s^2/n_s + sd_l^2/n_l)  (SE of mean difference)
    """
    mu_s, sigma_s = parse_stats(static_stats)
    mu_l, sigma_l = parse_stats(lamarckian_stats)

    delta = mu_s - mu_l
    sigma_delta = math.sqrt(sigma_s**2 + sigma_l**2)
    normalized = delta / mu_s if mu_s != 0 else float("nan")
    z_score = delta / sigma_delta if sigma_delta > 0 else float("inf")

    out = {
        "delta_mean": delta,
        "delta_std": sigma_delta,
        "normalized_improvement": normalized,
        "z_score": z_score,
    }

    if n_static is not None and n_lam is not None and n_static > 0 and n_lam > 0:
        delta_se = math.sqrt((sigma_s**2)/n_static + (sigma_l**2)/n_lam)
        out["delta_se"] = delta_se

    return out

---

In [ ]:
static = "Combined Mean: 177.8911 sec, Std Dev: 0.5546"
lam    = "Combined Mean: 166.5051 sec, Std Dev: 0.5298"

res = compare_policies(static, lam)

res = compare_policies(static, lam, n_static=3, n_lam=3)
for k, v in res.items():
    print(f"{k}: {v:.4f}")

delta_mean: 11.3860
delta_std: 0.7670
normalized_improvement: 0.0640
z_score: 14.8451
delta_se: 0.4428


---

### Speedup Analysis
---

In [ ]:
import re
import math
from typing import Dict, Tuple

def parse_time_stats(s: str) -> Tuple[float, float]:
    """
    Parse strings like:
    'Combined Mean: 29.5141 sec, Std Dev: 0.7580'
    Returns (mean, std_dev)
    """
    mean_match = re.search(r"Mean:\s*([0-9.]+)", s)
    std_match  = re.search(r"Std Dev:\s*([0-9.]+)", s)

    if mean_match is None or std_match is None:
        raise ValueError(f"Could not parse stats from: {s}")

    return float(mean_match.group(1)), float(std_match.group(1))


def compute_speedups(
    stats: Dict[str, str],
    runs: Dict[str, int],
    baseline: str = "Exhaustive",
):
    """
    stats: dict of method -> stats string
    runs:  dict of method -> number of runs
    baseline: method name used as denominator

    Prints speedup S_i = T_baseline / T_i and SE(S_i).
    """
    if baseline not in stats:
        raise ValueError("Baseline method not found in stats.")

    # Parse baseline
    T_exh, sd_exh = parse_time_stats(stats[baseline])
    n_exh = runs[baseline]
    se_exh = sd_exh / math.sqrt(n_exh)

    print(f"Baseline: {baseline}")
    print("-" * 60)
    print(f"{'Method':<15} {'S_i':>10} {'SE(S_i)':>12}")
    print("-" * 60)

    for method, s in stats.items():
        if method == baseline:
            print(f"{method:<15} {1.0:>10.3f} {'-':>12}")
            continue

        T_i, sd_i = parse_time_stats(s)
        n_i = runs[method]
        se_i = sd_i / math.sqrt(n_i)

        # Speedup
        S = T_exh / T_i

        # SE via delta method
        se_S = S * math.sqrt(
            (se_exh / T_exh) ** 2 +
            (se_i   / T_i)   ** 2
        )

        print(f"{method:<15} {S:>10.3f} {se_S:>12.3f}")

---

In [ ]:
stats = {
    "VP_MWV":     "Combined Mean: 3433.3767 sec, Std Dev: 102.9536",
    "Exhaustive": "Combined Mean: 18276.1617 sec, Std Dev: 16.8884",
    "Log":        "Combined Mean: 177.8911 sec, Std Dev: 0.5546",
    "Lamarckian": "Combined Mean: 166.5051 sec, Std Dev: 0.5298",
}

runs = {
    "VP_MWV": 3,
    "Exhaustive": 3,
    "Log": 3,
    "Lamarckian": 3,
}

compute_speedups(stats, runs)

Baseline: Exhaustive
------------------------------------------------------------
Method                 S_i      SE(S_i)
------------------------------------------------------------
VP_MWV               5.323        0.092
Exhaustive           1.000            -
Log                102.738        0.193
Lamarckian         109.763        0.210
